______

<img src="./imgs/DRRlogo.jpg" width="350" />

# Land Cover Classification for use in the CAPRA Model 

## Derivation of Additional Information 

_____

### Learning Objectives

In this lesson you will learn how to implement spectral indices and textural features that can be used to enhance phenomena of interest in remotely sensed images. 


### Spectral Indices

Spectral indices are based on the fact that reflectance spectra of different land covers are different. The indices are designed to exploit these differences to accentuate particular land cover types. Consider the following chart  of reflectance spectra for various targets:

<img src="./imgs/Spectral_Indices.PNG">


Observe that the land covers are separable at one or more wavelengths. Note, in particular, that vegetation curves have relatively high reflectance in the near-infrared (NIR) range, where radiant energy is scattered by cell walls. Also note that vegetation has low reflectance in the red range, where radiant energy is absorbed by chlorophyll. These observations motivate the formulation of vegetaion indices. 

Extracting spectral or spatial information is not a uncommon task during processing remotely sensed images because these information may improve classification results capturing additional information than spectral bands. <br>

Indices combine data from two or more spectral bands to accentuate land formation features.  Several Vegetation Indices (VIs) have been derived for Landsat data (Jensen, 2005). These indices are intended to enhance the vegetation signal, while minimizing any background effects or noise.  VIs can be calculated from brightness values (BVs), radiance values (Lλ) and reflectance values (ρp).  If vegetation transformations or indices are going to be used in spectral extraction for land cover classification purposes it is recommended to use reflectance values.

### Normalized Difference Vegetation Index

The Normalized Difference Vegetation Index (NDVI) is a quick and accurate way of analyzing vegetation in an image.  This calculation takes advantage of the fact that vegetation is very absorbent in the red band and very reflective in the NIR.  Therefore, surfaces with large amounts of vegetation can be found by comparing the ratio of red and NIR bands.

The output of NDVI is a new grid layer.  Higher values (above zero) signify a larger difference between the red and near infrared radiation recorded by the sensor - a condition associated with highly photosynthetically-active vegetation. Low NDVI values (less than zero) mean there is little difference between the red and NIR signals.  This is an indicator for presence of little photosynthetic activity, or when there is just very low NIR light reflectance (i.e., water reflects very low NIR light).

Google Earth Engine (GEE) has implemented a *normalizedDifference* function to compute the normalized difference between two bands. This function is used to compute the normalized vegetation index - [NDVI] (https://en.wikipedia.org/wiki/Normalized_Difference_Vegetation_Index). Given *B4* and  *B5*.

The normalized difference vegetation index ($NDVI$) is a band ratio that is related to the amount of green vegetation. 

\begin{equation*}
NDVI = \frac{NIR — RED}{NIR + RED} = \frac{Band5 — Band4}{Band5 + Band4}
\end{equation*}

where $NIR$ is the near infrared band and $RED$ is red band.



In [1]:
# import necessary packages
import ee
import numpy as np
from IPython.display import Image
from IPython.display import display
import pprint

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm, colorbar
from matplotlib import colors as mcolors

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()


In [2]:
# Input data
area_of_interest = ee.Geometry.Rectangle([-70.81, 19.76, -69.27, 18.44])
landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-01-01', '2018-04-19').min()
landsat8_collection = landsat8_collection.slice(0,5)
pprint.pprint(landsat8_collection.getInfo())

image = landsat8_collection.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':0,'max':2048,'size':'800', 'bands': 'B4,B3,B2', 'region':theGeom})
Image(url=thumbnail)

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B1'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B2'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B3'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            'data_type': {'m

In [3]:
# compute the normalized vegetation index using Infrared and red bands
# NDVI = (NIR - red) / (NIR + red)

# Functions to derive vegetation indices and other raster operations
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

ndvi = NDVI(landsat8_collection)

#Display thumbnail of NDVI output
image_ndvi = ndvi.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
#thumbnail = image_ndvi.getThumbUrl({'min':-1,'max':1,'size':'800'})
#Image(url=thumbnail)

clas_col = ','.join(['d7191c','fdae61','ffffbf','a6d96a','1a9641'])
Image(url = image_ndvi.getThumbUrl({'min': -1, 'max': 1, 'palette':clas_col}))

#### Questions:<br> (1)Examine the values in vegetated and water areas in the NDVI image. Where is the NDVI image darkest, why?<br>  (2)	If you were a farmer would you want a high or a low NDVI for your fields?<br>  

Other vegetation indices may involve additional computation rather than a normalized difference.<br>
For those cases, GEE includes the *expression* function. The following example calculates the *Enhanced Vegetation Index - EVI*

### Enhanced Vegetation Index

The Enhanced Vegetation Index (EVI) is designed to minimize saturation and background effects in NDVI. Since it is not a normalized difference index, we can compute it with the following expression:

\begin{equation*}
EVI = \frac{NIR — RED}{NIR + C1 * RED - C2 * BLUE + L} =  \frac{Band5 — Band4}{Band5 + C1 * Band4 - C2 * Band2 + L}
\end{equation*}

Where L is the canopy background adjustment that addresses non-linear, differential NIR and red radiant transfer through a canopy, and C1, C2 are the coefficients of the aerosol resistance term, which uses the blue band to correct for aerosol influences in the red band. The coefficients adopted in the Landsat 8 (OLI) algorithm are; L=1, C1 = 6, C2 = 7.5, and G (gain factor) = 2.5. 

Whereas the NDVI is chlorophyll sensitive, the EVI is more responsive to canopy structural variations, including leaf area index (LAI), canopy type, plant physiognomy, and canopy architecture. The two vegetation indices complement each other in global vegetation studies and improve upon the detection of vegetation changes and extraction of canopy biophysical parameters.

Another difference between Normalized Difference Vegetation Index (NDVI) and EVI is that in the presence of snow, NDVI decreases, while EVI increases (Huete, 2002).

In [4]:
#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

evi = EVI(landsat8_collection)

#Display thumbnail of classification output
image_evi = evi.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
#thumbnail = image_evi.getThumbUrl({'min':-1,'max':1,'size':'800','region':theGeom})
#Image(url=thumbnail)

clas_col = ','.join(['d7191c','fdae61','ffffbf','a6d96a','1a9641'])
Image(url = image_evi.getThumbUrl({'min': -1, 'max': 1, 'palette':clas_col}))


### Normalized Difference Water Index

The Normalized Difference Water Index (NDWI) was developed by Gao (1996) as an index of vegetation water content:

\begin{equation*}
NDWI = \frac{NIR — SWIR}{NIR + SWIR} = \frac{Band5 — Band3}{Band5 + Band3}
\end{equation*}

Note that this is not an exact implementation of NDWI according to the Landsat 8 Operational Land Imager (OLI) spectral response, since the OLI does not have a band in the right position (1.26 um).

You may try to compute the *NDWI*


## Textural Features 


When a small area in an image has a wide variation of shades of gray, the dominant property of that area is texture. Textural features provide information about the spatial distribution of the spectral information in an image band. Textural features may be not intuitive to the human eye but several studies have showed their contribution in image classification. Texture can be incorporated into a classification process by creating a new texture image that can be used as another feature. Each pixel in the new image has a brightness value that represents the texture at that location. <br> 

We will use the image.reduceNeighborhood() function to apply a reducer using a sliding window over the input image. The window size and shape will be specified by the ee.kernel function (tutorial 4). The output will be another image (in our case bands 1 to 5), with each new pixel representing the output of the reduction in a neighborhood around that pixel in the input image. 

We will select a square kernel that has the following parameters: <br>
+ *radius (Float)* :  The radius of the kernel to generate. For a 3x3 kernel, we should use 1, for 5x5 kernel we should use 2 and so on. <br>
+ *units (String) *:The system of measurement for the kernel "meters" or "pixels". Default value: "pixels".<br>
+ *normalize (Boolean)*: Indicates if the kernel values should be normalized to sum 1. Default value: "True".<br>
+ *magnitude (Float) *: Scale each value by this amount. Default: 1.<br>

In [5]:
# Define the kernel
kernel = ee.Kernel.square(1, 'pixels')  # filter mean of 3*3
# Compue mean as texture of the landsat 8 collection
mean = landsat8_collection.reduceNeighborhood(
    ee.Reducer.mean(),
    kernel
)
pprint.pprint(meanB1.getInfo())

NameError: name 'meanB1' is not defined

<b>Visualize textural features </b>

Minimun and maximum values of each band must be explored to set the visualization parameters.<br>
To find those values we will apply a <i>reducer</i> over the whole image. <br>


In [6]:
# Compute min and max of any band 
print(mean.select('B2_mean').reduce(ee.Reducer.minMax()).getInfo())

{'type': 'Image', 'bands': [{'id': 'min', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -32768.0, 'max': 32767.0}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'max', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -32768.0, 'max': 32767.0}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [7]:
thumbnail = landsat8_collection.select('B2').getThumbUrl({'min':0,'max':2000,'size':'800','region':theGeom})
Image(url=thumbnail)

In [8]:
thumbnail = mean.select('B2_mean').getThumbUrl({'min':0,'max':2048,'size':'800','region':theGeom})
Image(url=thumbnail)